# Czujnik dymu IoT - Eksploracja danych i uczenie maszynowe
Ninejszy raport przedstawia proces eksploracji danych oraz wynik budowy modelu w postaci efektów uczenia maszynowego, które zostały oparte na zbiorze danych [Smoke Detection Dataset](https://www.kaggle.com/datasets/deepcontractor/smoke-detection-dataset)
dalej zwanych danymi. Dane pochodzą z odczytów danych zmierzonych przez czujniki dymu typu IoT. Odczyty były wykonywane z częstotliwością $1Hz$. Czujniki dymu, z których pochodzą odczyty, były usytuowane w wielu warunkach środowiskowych począwszy od
wnętrz pomieszczeń, na zewnątrz, w sąsiedztwie z generatorami ciepła oraz dymu takimi jak ogniska, grille itp.

## Cel uczenia maszynowego
Zamierzonym celem było osiągnięcie modelu, umożliwiającego wykrywanie pożaru na podstawie wybranych zmiennych ilościowych. Wybór zmiennych ilościowych został dokonany na podstawie analizy danych ze względu na korelację.
Zmienna celu została zdefinowana jako zmienna opisowa dychotomiczna, która przyjmuje wartości $0$ i $1$ kolejno oznaczające `spoczynek` oraz `akcję uruchomienia alarmu/wykrycia pożaru`.

## Wczytanie wykorzystanych pakietów

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# uczenie maszynowe

#ignorowanie błędów
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'missingno'

## Wczytanie oraz przedstawienie danych

In [2]:
smoke_detection_iot_file = 'smoke_detection_iot.csv'
data = pd.read_csv(smoke_detection_iot_file)

In [5]:
data.shape

(62630, 16)

W danych znajduje się ponad 62 tysiące rekordów, każdy rozłożony na 16 kolumnt. Ilość wierszy przeważa nad ilością kolumn.

In [3]:
data.head()

,Unnamed: 0,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
0,0,1654733331,20.000,57.36,0,400,12306,18520,939.735,0.0,0.0,0.0,0.0,0.0,0,0
1,1,1654733332,20.015,56.67,0,400,12345,18651,939.744,0.0,0.0,0.0,0.0,0.0,1,0
2,2,1654733333,20.029,55.96,0,400,12374,18764,939.738,0.0,0.0,0.0,0.0,0.0,2,0
3,3,1654733334,20.044,55.28,0,400,12390,18849,939.736,0.0,0.0,0.0,0.0,0.0,3,0
4,4,1654733335,20.059,54.69,0,400,12403,18921,939.744,0.0,0.0,0.0,0.0,0.0,4,0


In [7]:
data.columns

Index(['Unnamed: 0', 'UTC', 'Temperature[C]', 'Humidity[%]', 'TVOC[ppb]',
       'eCO2[ppm]', 'Raw H2', 'Raw Ethanol', 'Pressure[hPa]', 'PM1.0', 'PM2.5',
       'NC0.5', 'NC1.0', 'NC2.5', 'CNT', 'Fire Alarm'],
      dtype='object')

Dane zostały podzielone na następujące zmienne opisowe:
* `Unnamed: 0` - licza porządkowa
* `UTC` - czas odczytu rekordu danych zebranych z czujnika dymu podany w formacie Coordinated Universal Time, UTC
* `Temperature[C]` - temperatura otoczenia czujnika dymu podana w stopniach Celsjusza
* `Humidity[%]` - wilgotność otoczenia czujnika dymu podana w procentach (wilgotność względna - wyrażony w procentach stosunek ciśnienia cząstkowego pray wodnej zawartej w powietrzu do prężności pary wodnej nasyconej nad płaską powierzchnią czystej wody w tej samej temperaturze [[źródło](https://pl.wikipedia.org/wiki/Wilgotno%C5%9B%C4%87_powietrza)])
* `TVOC[ppb]` - (***Total Volatile Organic Compounds***) *całkowita ilość lotnych związków organicznych* podana w ***parts per billion*** (***VOCs*** (*LZO*) - związki organiczne o wysokiej prężności pray [[źródło 1](https://pl.wikipedia.org/wiki/Lotne_zwi%C4%85zki_organiczne) [źródło 2](https://en.wikipedia.org/wiki/Volatile_organic_compound)]
* `eCO2[ppm]` - ekwiwalent koncentracji CO2 podany w ***parts per million***
* `Raw H2` - ilość wodóru molekularnego
* `Raw Ethanol` - ilość etanolu
* `Pressure[hPa]` - ciśnienie otoczenia czujnika wyrażone w hektopaskalach
*  `PM1.0` - ilość wykrytych cząstek o rozmiarze $<1.0\micro m$
*  `PM2.5` - ilość wykrytych cząstek o rozmiarze $>1.0\micro m$ i $<2.5\micro m$
*  `NC0.5` - koncentracja wykrtych cząstek o rozmiarze $<0.5\micro m$
*  `NC1.0` - koncentracja wykrtych cząstek o rozmiarze $>0.5\micro m$ i $<1.0\micro m$
*  `NC2.5` - koncentracja wykrtych cząstek o rozmiarze $>1.0\micro m$ i $<2.5\micro m$
*  `CNT` - licznik ilości zebranych pomiarów z danego czujnika dymu
*  `Fire Alarm` - stan czujnika dymu; $1$ jeśli czujnik wykrywa "pożar" (jest uruchamiany alarm) - jest to zmienna celu

Po zapoznaniu się z danymi uznano, że kolumny `Unnamed: 0`, `UTC` oraz `CNT` nie są skorelowane ze zmienna celu. W rezultacie kolumny te pominęto w dalszej analizie oraz usunięto je ze zbioru.

In [13]:
significantColumns = ['Temperature[C]', 'Humidity[%]', 'TVOC[ppb]', 'eCO2[ppm]', 'Raw H2', 'Raw Ethanol', 'Pressure[hPa]', 'PM1.0', 'PM2.5', 'NC0.5', 'NC1.0', 'NC2.5', 'Fire Alarm']
data = data[significantColumns]
data.shape

(62630, 13)

In [14]:
data.describe()

,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,Fire Alarm
count,62630.000000,62630.000000,62630.000000,62630.000000,62630.000000,62630.000000,62630.000000,62630.000000,62630.000000,62630.000000,62630.000000,62630.000000,62630.000000
mean,15.970424,48.539499,1942.057528,670.021044,12942.453936,19754.257912,938.627649,100.594309,184.467770,491.463608,203.586487,80.049042,0.714626
std,14.359576,8.865367,7811.589055,1905.885439,272.464305,609.513156,1.331344,922.524245,1976.305615,4265.661251,2214.738556,1083.383189,0.451596
min,-22.010000,10.740000,0.000000,400.000000,10668.000000,15317.000000,930.852000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.994250,47.530000,130.000000,400.000000,12830.000000,19435.000000,938.700000,1.280000,1.340000,8.820000,1.384000,0.033000,0.000000
50%,20.130000,50.150000,981.000000,400.000000,12924.000000,19501.000000,938.816000,1.810000,1.880000,12.450000,1.943000,0.044000,1.000000
75%,25.409500,53.240000,1189.000000,438.000000,13109.000000,20078.000000,939.418000,2.090000,2.180000,14.420000,2.249000,0.051000,1.000000
max,59.930000,75.200000,60000.000000,60000.000000,13803.000000,21410.000000,939.861000,14333.690000,45432.260000,61482.030000,51914.680000,30026.438000,1.000000
